In [1]:
import pandas as pd 
import evalml 
from evalml.automl import AutoMLSearch


data = pd.read_excel('c:/users/omar/Real estate valuation data set.xlsx', header=0)
#data.head()
data.drop('No', axis=1, inplace=True)
A = data.iloc[:,:-1].copy()
b = data.iloc[:,-1].copy()
A.head()


,X1 transaction date,X2 house age,X3 distance to the nearest MRT station,X4 number of convenience stores,X5 latitude,X6 longitude
0,2012.916667,32.0,84.87882,10,24.98298,121.54024
1,2012.916667,19.5,306.59470,9,24.98034,121.53951
2,2013.583333,13.3,561.98450,5,24.98746,121.54391
3,2013.500000,13.3,561.98450,5,24.98746,121.54391
4,2012.833333,5.0,390.56840,5,24.97937,121.54245


In [2]:
from evalml.pipelines import RegressionPipeline

LinearRegressionPipeline= RegressionPipeline(component_graph= [ 'Standard Scaler', 'Linear Regressor'])
    
#component_graph_SVR = {
#    'StandardScaler': ['Standard Scaler'],
#    'Support Vector Regressor': ['SVM Regressor', 'Standard Scaler']
#}
    
SVMRegressionPipeline = RegressionPipeline(component_graph=[ 'Standard Scaler','SVM Regressor'])

#component_graph_RFR = {
#    'StandardScaler': ['Standard Scaler'],
#    'Random Forest Regressor': ['Random Forest Regressor', 'Standard Scaler']
#}

    
RFRegressionPipeline = RegressionPipeline(component_graph = ['Standard Scaler','Random Forest Regressor'] )
    

In [3]:
import woodwork as ww
x = ww.DataTable(A, semantic_tags= {'X1 transaction date':'timedelta64'}, 
                 logical_types={'X1 transaction date':'double'})
y = ww.DataColumn(b)

x_train, x_test, y_train, y_test = evalml.preprocessing.split_data(x,y,problem_type= 'regression')



In [4]:
automl = AutoMLSearch(x_train,y_train, problem_type='regression',
                     objective = 'MAE', additional_objectives = ['R2','MSE'],
                      allowed_pipelines = [LinearRegressionPipeline,SVMRegressionPipeline,RFRegressionPipeline]
                     )
automl.search()


Using default limit of max_batches=1.

3 pipelines ready for search.

*****************************
* Beginning pipeline search *
*****************************

Optimizing for MAE. 
Lower score is better.

Using SequentialEngine to train and score pipelines.
Searching up to 1 batches for a total of 4 pipelines. 
Allowed model families: random_forest, svm, linear_model



FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Best Score',
              'type'…

Evaluating Baseline Pipeline: Mean Baseline Regression Pipeline
Mean Baseline Regression Pipeline:
	Starting cross validation
	Finished cross validation - mean MAE: 10.857

*****************************
* Evaluating Batch Number 1 *
*****************************

Linear Regressor w/ Standard Scaler:
	Starting cross validation
	Finished cross validation - mean MAE: 6.464
SVM Regressor w/ Standard Scaler:
	Starting cross validation
	Finished cross validation - mean MAE: 6.282
Random Forest Regressor w/ Standard Scaler:
	Starting cross validation
	Finished cross validation - mean MAE: 4.873

Search finished after 00:03            
Best pipeline: Random Forest Regressor w/ Standard Scaler
Best pipeline MAE: 4.873406


In [5]:
#Generate a pipeline

best_pipeline = automl.best_pipeline

automl.describe_pipeline(automl.rankings.iloc[0]["id"])



**********************************************
* Random Forest Regressor w/ Standard Scaler *
**********************************************

Problem Type: regression
Model Family: Random Forest

Pipeline Steps
1. Standard Scaler
2. Random Forest Regressor
	 * n_estimators : 100
	 * max_depth : 6
	 * n_jobs : -1

Training
Training for regression problems.
Total training time (including CV): 1.8 seconds

Cross Validation
----------------
              MAE    R2    MSE # Training # Validation
0           5.280 0.637 90.787        220          111
1           4.258 0.754 37.940        221          110
2           5.082 0.652 51.323        221          110
mean        4.873 0.681 60.017          -            -
std         0.542 0.064 27.476          -            -
coef of var 0.111 0.094  0.458          -            -
